# Differential Uplift Analysis at Axial Seamount

**Author:** Dax Soule  
**Date:** January 2026  
**Data Period:** 2015-01-01 to 2026-01-16

---

## Overview

This notebook calculates **differential uplift** between two Bottom Pressure Recorders (NANO-BPRs) at Axial Seamount, an active submarine volcano on the Juan de Fuca Ridge. By comparing pressure measurements from two locations within the caldera, we can track volcanic deformation independently of regional oceanographic signals.

### Scientific Background

Axial Seamount is one of the most active submarine volcanoes in the world, with eruptions recorded in 1998, 2011, and 2015. Bottom Pressure Recorders (BPRs) measure the water column pressure above the seafloor, which changes as the seafloor moves up (inflation) or down (deflation) due to magma accumulation or withdrawal.

**Differential uplift** (the difference between two BPRs) removes common-mode signals like:
- Ocean tides
- Atmospheric pressure changes
- Regional oceanographic variability

This leaves only the local volcanic deformation signal.

### Stations

| Station | Location | Description |
|---------|----------|-------------|
| MJ03E | Eastern Caldera | Reference station |
| MJ03F | Central Caldera | Near the eruptive vent |

The differential signal (MJ03E - MJ03F) shows relative vertical displacement between these two points.

## Setup

### Environment

To reproduce this analysis, create a conda environment from the provided `environment.yml`:

```bash
conda env create -f environment.yml
conda activate axial-bpr-analysis
```

### Imports

In [ ]:
import re
from pathlib import Path

import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Display settings
pd.set_option('display.max_columns', 10)
%matplotlib inline

### Configuration

Define data paths and analysis parameters. 

**Note:** You will need to update `MJ03E_PATH` and `MJ03F_PATH` to point to your local copies of the OOI data.

In [ ]:
# Data paths - UPDATE THESE FOR YOUR SYSTEM
MJ03E_PATH = Path("/home/jovyan/ooi/kdata/RS03ECAL-MJ03E-06-BOTPTA302-streamed-botpt_nano_sample_15s")
MJ03F_PATH = Path("/home/jovyan/ooi/kdata/RS03CCAL-MJ03F-05-BOTPTA301-streamed-botpt_nano_sample_15s")

# Output directory
OUTPUT_DIR = Path(".")

# Time range for analysis
TIME_START = "2015-01-01"
TIME_END = "2026-01-16"
TIME_START_YEAR = 2015
TIME_END_YEAR = 2026

print(f"Analysis period: {TIME_START} to {TIME_END}")
print(f"MJ03E data path: {MJ03E_PATH}")
print(f"MJ03F data path: {MJ03F_PATH}")

---

## Data Loading

### Pressure-to-Depth Conversion

The BPR measures pressure in **psia** (pounds per square inch absolute). We convert this to depth in meters using a simplified linear relationship:

$$\text{depth}_m = (\text{pressure}_{psia} - 14.7) \times 0.670$$

Where:
- 14.7 psia is atmospheric pressure (subtracted to get gauge pressure)
- 0.670 m/psi is an approximate conversion factor for seawater at this depth

**Note:** This is an approximation. A more rigorous approach would account for salinity, temperature, and the equation of state for seawater. However, for differential measurements, these factors largely cancel out.

In [ ]:
def pressure_to_depth(pressure_psia):
    """Convert pressure in psia to depth in meters.
    
    Parameters
    ----------
    pressure_psia : array-like
        Pressure in pounds per square inch absolute
        
    Returns
    -------
    array-like
        Depth in meters
    """
    return (pressure_psia - 14.7) * 0.670

### File Filtering

The OOI data archive contains multiple file types. We specifically want the **15-second sample** files (`_15s_`) and only files that overlap with our target time range.

In [ ]:
def filter_files_by_time_range(nc_files: list[Path]) -> list[Path]:
    """Filter NetCDF files to those covering the target time range.
    
    Only includes 15-second sample files that overlap with TIME_START to TIME_END.
    
    Parameters
    ----------
    nc_files : list of Path
        List of NetCDF file paths
        
    Returns
    -------
    list of Path
        Filtered and sorted list of file paths
    """
    filtered = []
    # Pattern to extract start and end years from filename
    pattern = re.compile(r"_15s_(\d{4})\d{4}T\d{6}-(\d{4})\d{4}T")

    for f in nc_files:
        # Only include 15s sample files
        if "_15s_" not in f.name:
            continue

        match = pattern.search(f.name)
        if match:
            start_year = int(match.group(1))
            end_year = int(match.group(2))
            # Include file if it overlaps with our time range
            if start_year <= TIME_END_YEAR and end_year >= TIME_START_YEAR:
                filtered.append(f)

    return sorted(filtered)

### Station Data Loading

Each station's data is loaded file-by-file to manage memory (the full dataset is large). We:

1. Load each NetCDF file
2. Swap dimensions from `obs` to `time` for proper time-series handling
3. Filter to our target time range
4. Convert pressure to depth
5. Resample from 15-second to **hourly** averages (reduces data volume by ~240x)
6. Concatenate all chunks and remove duplicates

In [ ]:
def load_station(data_path: Path, station_name: str) -> pd.Series:
    """Load pressure data for a station, filtered to time range, resampled to hourly.
    
    Parameters
    ----------
    data_path : Path
        Directory containing NetCDF files for this station
    station_name : str
        Name of station (for logging)
        
    Returns
    -------
    pd.Series
        Hourly depth time series with DatetimeIndex
    """
    all_files = sorted(data_path.glob("*.nc"))
    nc_files = filter_files_by_time_range(all_files)

    print(f"{station_name}: Loading {len(nc_files)} files")

    hourly_chunks = []

    for i, f in enumerate(nc_files):
        if i % 50 == 0:  # Progress update every 50 files
            print(f"  Processing file {i+1}/{len(nc_files)}...")

        # Load single file
        ds = xr.open_dataset(f, engine="netcdf4")
        
        # OOI data uses 'obs' as the dimension; swap to 'time' for time-series operations
        ds = ds.swap_dims({"obs": "time"})

        # Filter to target time range
        ds = ds.sel(time=slice(TIME_START, TIME_END))

        if len(ds.time) == 0:
            ds.close()
            continue

        # Get pressure, convert to depth, resample to hourly (reduces memory)
        pressure = ds["bottom_pressure"].values
        time = ds["time"].values
        ds.close()

        # Create series and resample
        depth = pressure_to_depth(pressure)
        series = pd.Series(depth, index=pd.DatetimeIndex(time))
        hourly = series.resample("1h").mean()
        hourly_chunks.append(hourly)

    # Concatenate all chunks
    result = pd.concat(hourly_chunks).sort_index()
    result = result[~result.index.duplicated(keep="first")]

    print(f"{station_name}: {len(result)} hourly observations")
    return result

### Load Both Stations

**Warning:** This cell takes several minutes to run as it processes hundreds of NetCDF files.

In [ ]:
print("Loading MJ03E (Eastern Caldera)...")
depth_e = load_station(MJ03E_PATH, "MJ03E")

print("\nLoading MJ03F (Central Caldera)...")
depth_f = load_station(MJ03F_PATH, "MJ03F")

In [ ]:
# Quick look at the data
print("MJ03E depth range:", depth_e.min(), "to", depth_e.max(), "meters")
print("MJ03F depth range:", depth_f.min(), "to", depth_f.max(), "meters")
print("\nTime range:", depth_e.index.min(), "to", depth_e.index.max())

---

## Quality Control: Spike Removal

### The Problem

BPR data contains occasional **spikes** - brief, anomalous readings caused by sensor glitches, electrical interference, or data transmission errors. These appear as sudden jumps that don't represent real seafloor motion.

### Why MAD Instead of Standard Deviation?

A common approach is to flag values more than 3 standard deviations from a rolling mean. However, **standard deviation is sensitive to outliers** - the very spikes we're trying to detect inflate the standard deviation, making them harder to catch.

**Median Absolute Deviation (MAD)** is a robust alternative:

$$\text{MAD} = \text{median}(|X_i - \text{median}(X)|)$$

Because MAD uses medians instead of means, outliers have minimal influence on the threshold calculation.

### Scaling Factor

For normally distributed data, the relationship between standard deviation and MAD is:

$$\sigma \approx 1.4826 \times \text{MAD}$$

We use this scaling factor so our threshold is interpretable in terms of "sigma" equivalents.

### Algorithm

1. Calculate a **24-hour centered rolling median** (robust central tendency)
2. Compute the absolute deviation of each point from this rolling median
3. Calculate a **rolling MAD** from these deviations
4. Scale MAD by 1.4826 to approximate standard deviation
5. Flag points where deviation > threshold × scaled_MAD
6. Replace flagged points with NaN

In [ ]:
def remove_spikes(series: pd.Series, window_hours: int = 24, threshold: float = 5.0) -> pd.Series:
    """Remove spikes using rolling median and MAD (median absolute deviation).

    MAD is more robust to outliers than standard deviation.

    Parameters
    ----------
    series : pd.Series
        Hourly depth time series
    window_hours : int
        Rolling window size in hours (default: 24)
    threshold : float
        Number of scaled MADs for spike threshold (default: 5.0)

    Returns
    -------
    pd.Series
        Series with spikes replaced by NaN
    """
    cleaned = series.copy()

    # Use median (robust to outliers)
    rolling_median = cleaned.rolling(window=window_hours, center=True, min_periods=1).median()

    # Calculate MAD (median absolute deviation) - more robust than std
    deviation = (cleaned - rolling_median).abs()
    rolling_mad = deviation.rolling(window=window_hours, center=True, min_periods=1).median()

    # Scale MAD to be comparable to std (for normal distribution, std ≈ 1.4826 * MAD)
    scaled_mad = 1.4826 * rolling_mad

    # Flag values more than threshold MADs from rolling median
    is_spike = deviation > (threshold * scaled_mad)

    n_spikes = is_spike.sum()
    if n_spikes > 0:
        print(f"    Removed {n_spikes} spikes ({100*n_spikes/len(series):.2f}%)")
        cleaned[is_spike] = pd.NA

    return cleaned

### Apply Spike Removal to Individual Stations

We use a **conservative threshold of 5.0** for individual station data to catch only obvious sensor glitches.

In [ ]:
print("Removing spikes from MJ03E...")
depth_e_clean = remove_spikes(depth_e, window_hours=24, threshold=5.0)

print("Removing spikes from MJ03F...")
depth_f_clean = remove_spikes(depth_f, window_hours=24, threshold=5.0)

---

## Computing Differential Uplift

### Alignment and Calculation

The two stations don't always have simultaneous measurements (due to data gaps, maintenance, etc.). We:

1. Combine both series into a DataFrame
2. Drop rows where either station is missing
3. Calculate the difference: **MJ03E - MJ03F**

### Interpretation

- **Positive differential** = MJ03E is deeper than MJ03F = Central Caldera (MJ03F) has uplifted relative to Eastern Caldera
- **Negative differential** = MJ03E is shallower = Central Caldera has subsided

In [ ]:
# Align on common time index
combined = pd.DataFrame({
    "depth_mj03e_m": depth_e_clean,
    "depth_mj03f_m": depth_f_clean
}).dropna()

# Calculate differential depth: MJ03E - MJ03F
combined["differential_m"] = combined["depth_mj03e_m"] - combined["depth_mj03f_m"]

print(f"Combined dataset: {len(combined)} hourly observations")
print(f"Time range: {combined.index.min()} to {combined.index.max()}")

### Spike Removal on Differential Signal

Some spikes only appear in the **differential** signal - when one sensor glitches but not the other. These aren't caught by filtering individual stations.

We use a **more aggressive threshold of 3.5** for the differential signal because:
1. The differential is inherently smoother (common-mode noise removed)
2. Spikes in the differential are more likely to be artifacts
3. We want to preserve real volcanic signals while removing obvious glitches

In [ ]:
print("Removing spikes from differential signal...")
combined["differential_m"] = remove_spikes(combined["differential_m"], window_hours=24, threshold=3.5)

### Daily Averaging

For visualization and analysis, we resample to **daily means**. This:
- Removes residual tidal signals
- Reduces noise
- Makes long-term trends clearer

In [ ]:
daily = combined.resample("1D").mean()
print(f"Daily dataset: {len(daily)} days")

---

## Visualization

### Individual Station Depths

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 6), sharex=True)

# MJ03E
axes[0].plot(daily.index, daily["depth_mj03e_m"], color="blue", linewidth=0.5)
axes[0].set_ylabel("Depth (m)")
axes[0].set_title("MJ03E - Eastern Caldera")
axes[0].invert_yaxis()
axes[0].grid(True, alpha=0.3)

# MJ03F
axes[1].plot(daily.index, daily["depth_mj03f_m"], color="red", linewidth=0.5)
axes[1].set_ylabel("Depth (m)")
axes[1].set_title("MJ03F - Central Caldera")
axes[1].set_xlabel("Year")
axes[1].invert_yaxis()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Differential Uplift Time Series

This is the key result showing volcanic deformation at Axial Seamount.

**Reference lines:**
- **Solid red**: The 2015 pre-eruption high (maximum differential before the April 2015 eruption)
- **Dashed red**: ±20 cm from the 2015 high (uncertainty/forecasting bounds)

In [ ]:
# Find 2015 high value for reference line
uplift_2015 = daily["differential_m"]["2015"]
high_2015 = uplift_2015.max()
print(f"2015 pre-eruption high: {high_2015:.2f} m")

In [ ]:
# Publication-quality figure
plt.rcParams.update({
    'font.size': 10,
    'axes.labelsize': 11,
    'axes.titlesize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
})

fig, ax = plt.subplots(figsize=(10, 4))

# Plot the data
ax.plot(daily.index, daily["differential_m"].values,
        color="#2E86AB", linewidth=1, label="Daily mean")

# Add red horizontal line for 2015 high
ax.axhline(y=high_2015, color="red", linestyle="-", linewidth=1.5,
           label=f"2015 high ({high_2015:.2f} m)")

# Add red dashed lines at +/- 20 cm from 2015 high
ax.axhline(y=high_2015 + 0.20, color="red", linestyle="--", linewidth=1, alpha=0.7)
ax.axhline(y=high_2015 - 0.20, color="red", linestyle="--", linewidth=1, alpha=0.7)

# Labels and title
ax.set_xlabel("Year")
ax.set_ylabel("Differential Depth (m)")
ax.set_title("Differential Uplift at Axial Seamount (MJ03E − MJ03F)")

# Format x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y"))
ax.xaxis.set_major_locator(mdates.YearLocator())

# Clean up
ax.spines["top"].set_visible(False)
ax.spines["right"].set_visible(False)
ax.grid(True, alpha=0.3, linestyle="-", linewidth=0.5)
ax.legend(loc="lower right", framealpha=0.9, fontsize=9)

plt.tight_layout()
plt.show()

# Reset rcParams
plt.rcParams.update(plt.rcParamsDefault)

### Interpretation

The plot shows:

1. **Early 2015**: Gradual inflation as magma accumulates beneath the caldera
2. **April 2015**: Sharp drop (~1.3 m) marking the eruption and magma withdrawal
3. **2015-2026**: Steady re-inflation as the magma chamber recharges
4. **~2022**: The differential crossed the 2015 pre-eruption high
5. **Present (2026)**: Now ~20 cm above the 2015 high, suggesting continued inflation

This pattern is consistent with the volcanic cycle at Axial Seamount: gradual inflation between eruptions, followed by rapid deflation during eruptions.

---

## Data Export

Export the cleaned data to Parquet format for use in other analyses.

In [ ]:
# Export hourly data
combined.to_parquet(OUTPUT_DIR / "differential_uplift_hourly.parquet")
print(f"Exported: differential_uplift_hourly.parquet ({len(combined)} rows)")

# Export daily data
daily.to_parquet(OUTPUT_DIR / "differential_uplift_daily.parquet")
print(f"Exported: differential_uplift_daily.parquet ({len(daily)} rows)")

### Loading Exported Data

To use this data in another analysis:

In [ ]:
# Example: loading and joining with another dataset
bpr = pd.read_parquet(OUTPUT_DIR / "differential_uplift_daily.parquet")
print(bpr.head())
print(f"\nColumns: {list(bpr.columns)}")
print(f"Index: {bpr.index.name} ({bpr.index.dtype})")

---

## Summary

This notebook demonstrated:

1. **Data loading** from OOI NetCDF archives with memory-efficient chunked processing
2. **Pressure-to-depth conversion** using a simplified linear relationship
3. **Quality control** using MAD-based spike detection (more robust than standard deviation)
4. **Differential uplift calculation** to isolate volcanic deformation from oceanographic signals
5. **Visualization** of the ~11-year volcanic inflation/deflation cycle
6. **Data export** to Parquet for cross-instrument analysis

### Key Findings

- Axial Seamount has been steadily inflating since the April 2015 eruption
- As of early 2026, the differential uplift has exceeded the 2015 pre-eruption level by ~20 cm
- The inflation rate appears roughly linear, suggesting continued magma recharge

### References

- Nooner, S. L., & Chadwick, W. W. (2016). Inflation-predictable behavior and co-eruption deformation at Axial Seamount. *Science*, 354(6318), 1399-1403.
- OOI Cabled Array: https://oceanobservatories.org/array/cabled-array/